In [6]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [7]:
url = 'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
dataset = tf.keras.utils.get_file('aclImdb_v1.tar.gz', url, untar=True, cache_dir='.', cache_subdir='')
dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')

84131840/84125825 [==============================] - 290s 3us/step


In [8]:
print(dataset_dir)

./aclImdb


In [9]:
train_dir = os.path.join(dataset_dir, 'train')
os.listdir(train_dir)

['urls_unsup.txt',
 'neg',
 'urls_pos.txt',
 'unsup',
 'urls_neg.txt',
 'pos',
 'unsupBow.feat',
 'labeledBow.feat']

In [10]:
fname = os.path.join(train_dir, 'pos/1181_9.txt')
with open(fname) as f:
    print(f.read())

Rachel Griffiths writes and directs this award winning short film. A heartwarming story about coping with grief and cherishing the memory of those we've loved and lost. Although, only 15 minutes long, Griffiths manages to capture so much emotion and truth onto film in the short space of time. Bud Tingwell gives a touching performance as Will, a widower struggling to cope with his wife's death. Will is confronted by the harsh reality of loneliness and helplessness as he proceeds to take care of Ruth's pet cow, Tulip. The film displays the grief and responsibility one feels for those they have loved and lost. Good cinematography, great direction, and superbly acted. It will bring tears to all those who have lost a loved one, and survived.


In [11]:
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

In [12]:
os.listdir(train_dir)

['urls_unsup.txt',
 'neg',
 'urls_pos.txt',
 'urls_neg.txt',
 'pos',
 'unsupBow.feat',
 'labeledBow.feat']

In [13]:
batch_size = 32
seed = 42

raw_train_ds = preprocessing.text_dataset_from_directory('aclImdb/train', batch_size=batch_size, validation_split=0.2, subset='training', seed=seed)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.


In [14]:
for text_batch, label_batch in raw_train_ds.take(1):
    for i in range(3):
        print('Review: ', text_batch.numpy()[i])
        print('Label: ', label_batch.numpy()[i])

Review:  b'"Pandemonium" is a horror movie spoof that comes off more stupid than funny. Believe me when I tell you, I love comedies. Especially comedy spoofs. "Airplane", "The Naked Gun" trilogy, "Blazing Saddles", "High Anxiety", and "Spaceballs" are some of my favorite comedies that spoof a particular genre. "Pandemonium" is not up there with those films. Most of the scenes in this movie had me sitting there in stunned silence because the movie wasn\'t all that funny. There are a few laughs in the film, but when you watch a comedy, you expect to laugh a lot more than a few times and that\'s all this film has going for it. Geez, "Scream" had more laughs than this film and that was more of a horror film. How bizarre is that?<br /><br />*1/2 (out of four)'
Label:  0
Review:  b"David Mamet is a very interesting and a very un-equal director. His first movie 'House of Games' was the one I liked best, and it set a series of films with characters whose perspective of life changes as they get

In [15]:
print(raw_train_ds.class_names[0])

neg


In [16]:
raw_val_ds = preprocessing.text_dataset_from_directory('aclImdb/train', batch_size=batch_size, seed=seed, validation_split=0.2, subset='validation')

Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [17]:
raw_test_ds = preprocessing.text_dataset_from_directory('aclImdb/test', batch_size=batch_size)

Found 25000 files belonging to 2 classes.


In [18]:
# standardization of text -- means remove html tags and punctuation
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_text = tf.strings.regex_replace(lowercase, '<br >', ' ')
    return tf.strings.regex_replace(stripped_text, '[%s]' % re.escape(string.punctuation), '')

In [23]:
max_features = 10000
sequence_length = 250

vectorize_layer = TextVectorization(standardize=custom_standardization, max_tokens=max_features, output_sequence_length=sequence_length, output_mode='int')

In [34]:
train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

In [35]:
print(len(vectorize_layer.get_vocabulary()))

10000


In [36]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label

In [37]:
text_batch, label_batch = next(iter(raw_train_ds))
first_review, first_label = text_batch[0], label_batch[0]
print(vectorize_text(first_review, first_label))

(<tf.Tensor: shape=(1, 250), dtype=int64, numpy=
array([[   4,  182,  246, 2308,   36,    2,    1,   12,   56,    7,    4,
        1820,  436,  715,    4,  219, 1740,    5,   34,    1, 3419,    5,
           3,  546,   16, 9049,  109,   29,  679,    6,   40,  493, 2245,
          33,  348,   56, 4092,    2, 4293,    1,    3,  842,    2,  348,
          56,  876, 1369,    8,  116,   17,    4,  237,  132,   40,  119,
         724,    6,  191,    4,  460,   16,    2,  125,   52,   56,    7,
        3394,    3,   40, 1535,    7,  546,   33,    4, 1107,    5, 3713,
        9072,  173,   33,  129, 2360,  649,    2,  246, 1744,    6,   40,
        1087,  779,    3, 4979, 1102,   21,    2, 3713, 1658,    6,    2,
         290,    1,    1,   13,    1,    1,    1,  982,    1,    1,  777,
          17,  373, 7733,    1,    1,  837,   33,    1, 1928,    1,    1,
          37,   15,    1,    1,    1, 2109,   56,    7,    4, 1820,    2,
           1,  171,    7,   52,    2,  378, 3890,  994,  986,  

In [38]:
len(vectorize_layer.get_vocabulary())

10000

In [39]:
train_ds = raw_train_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)

In [40]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [41]:
embedding_dim = 16

model = tf.keras.models.Sequential([
    layers.Embedding(max_features + 1, embedding_dim),
    layers.Dropout(0.2),
    layers.GlobalAveragePooling1D(),
    layers.Dropout(0.2),
    layers.Dense(1)
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          160016    
_________________________________________________________________
dropout (Dropout)            (None, None, 16)          0         
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 17        
Total params: 160,033
Trainable params: 160,033
Non-trainable params: 0
_________________________________________________________________


In [42]:
model.compile(loss=losses.BinaryCrossentropy(from_logits=True), optimizer='adam', metrics=tf.metrics.BinaryAccuracy(threshold=0.0))

In [43]:
epochs = 10
model.fit(train_ds, epochs=10, validation_data=val_ds)

Epoch 1/10
625/625 [==============================] - 22s 34ms/step - loss: 0.6839 - binary_accuracy: 0.5996 - val_loss: 0.6210 - val_binary_accuracy: 0.7652
Epoch 2/10
625/625 [==============================] - 4s 6ms/step - loss: 0.5858 - binary_accuracy: 0.7777 - val_loss: 0.5047 - val_binary_accuracy: 0.8168
Epoch 3/10
625/625 [==============================] - 3s 5ms/step - loss: 0.4725 - binary_accuracy: 0.8328 - val_loss: 0.4262 - val_binary_accuracy: 0.8442
Epoch 4/10
625/625 [==============================] - 3s 5ms/step - loss: 0.3973 - binary_accuracy: 0.8558 - val_loss: 0.3789 - val_binary_accuracy: 0.8574
Epoch 5/10
625/625 [==============================] - 3s 6ms/step - loss: 0.3491 - binary_accuracy: 0.8737 - val_loss: 0.3494 - val_binary_accuracy: 0.8648
Epoch 6/10
625/625 [==============================] - 3s 6ms/step - loss: 0.3164 - binary_accuracy: 0.8826 - val_loss: 0.3300 - val_binary_accuracy: 0.8698
Epoch 7/10
625/625 [==============================] - 4s 6ms/s

In [44]:
model.evaluate(test_ds)

782/782 [==============================] - 19s 24ms/step - loss: 0.3136 - binary_accuracy: 0.8716


[0.313599556684494, 0.8715599775314331]

In [48]:
export_model = tf.keras.models.Sequential([
    vectorize_layer,
    model,
    layers.Activation('sigmoid')
])
export_model.compile(optimizer='adam', loss=losses.BinaryCrossentropy(from_logits=False), metrics=['acc'])
loss, accuracy = export_model.evaluate(raw_test_ds)

782/782 [==============================] - 7s 9ms/step - loss: 0.3162 - acc: 0.8686
